# Introduction
This file ....


## Setup
Run this cell for the setups

In [ ]:
!git clone https://github.com/nicofirst1/TranscriptionTagger
!pip install nltk

Import necessary libraries

In [ ]:
import csv
import os
from utils import get_name, count_tokens
from Configs import Configs

configs=Configs()

# Corpus file
Finally, you need to upload your corpus file.


In [ ]:
# DONT USE THIS, IT IS JUST FOR DEBUG
if False:
    corpus_path = "/Users/giulia/Downloads/Telegram Desktop/input.txt"
    with open(corpus_path, "r+", encoding="utf16") as f:
        corpus_text = f.read()

In [ ]:
corpus_path = files.upload()

assert len(corpus_path) ==1 , "Support for multiple transcription file is not available! Please upload just one file"
corpus_text=list(corpus_path.values())[0]
corpus_path=list(corpus_path.keys())[0]

corpus_text=corpus_text.decode("utf16")

# print first 50 character of the transcription file
print("\n\n First 400 characters:\n")
corpus_text[:400]

## Output file
<br>
In order to obtain the comma separated values (`csv`) file, you can specify the separator. By default the separator value is a comma (as the name implies), but you can also use:<br>
- semicolon: `;`<br>
- comma: `,`<br>
- tab : `\t`<br>
<br>
In our project we use the tab, since it is the one that allows excel to view it (for Germany the default value is tab).

In [ ]:
separator = configs.separator

# Main program<br>
Time to start the main program. First let us merge all the variables we uploaded before

In [ ]:
# step 1: split the whole corpus in different elements every new line, creating a list of paragraphs. For us this means splitting interviewer and interviewee in different paragraphs
trans = corpus_text.split("\n")
# step 2: our first two paragraphs are the file name and the date, which we don't need so discard them
trans = trans[2:]
# step 3: remove spaces at the start and end of each paragraph
trans = [x.strip() for x in trans]
# step 4 : remove empty paragraphs from the list
trans = [x for x in trans if x != '']

# take a peek at the first four paragraphs
for idx in range(3):
    print(trans[idx])

### Interviewers and Interviewees
As you can see form the previous example, we consider a file where we have only one interviewer and one interviewee alternating each other. But it can be the case where you have multiple interviewers and interviewees in random order. In this case we need to know the names of the interviewers in order to split them from the interviewees.
If you need to add multiple interviewers run the following cell after adding the names


In [ ]:
# interviewers = "name1,name2,...,nameN"
interviewers = ""

Get the interviewers

In [ ]:
# if user specified the interviewer's names then take that
if len(interviewers) > 0:
    interviewers = interviewers.split(',')
    # remove spaces
    interviewers = [x.strip() for x in interviewers]
else:
    # else use the first character of the transcription
    interviewers = [trans[0][0]]
print(f"The selected interviewers are: {', '.join(interviewers)}")

### Output file settings<br>
Now it's time to create the output file.

The output file will have the same name as the transcription one but with `_analysis.csv` at the end

In [ ]:
# get the name of the output path
output_path = os.path.basename(corpus_path)
output_path = os.path.splitext(output_path)[0] + "_analysis.csv"
output_path

### Finding interviewees

This part looks for all the names present in the file

In [ ]:
# get interviewer/interviewees names

names = [get_name(x).strip() for x in trans]
names = set(names)

# remove all mention of interviwers in names

for i in interviewers:
    names = [x for x in names if i not in x]
interviewees = list(names)

# notify user about names

print(f"I found the following interviewees names: {', '.join(interviewees)}")

### Define the csv head

Our csv contains the following informations:
- the considered token (or word)
- how many times was repeated in all the corpus
- for each speaker, how many times it was repeated for that speaker

In [ ]:
speakers = interviewees + interviewers
csv_header = ['token', 'total repetitions'] + [f"repetitions in {x}" for x in speakers]
print(f"The csv header looks like this")
csv_header

We create the data for analysis

In [ ]:

speaker_specific_data = {k: [x.replace(k, "") for x in trans if get_name(x) == k] for k in speakers}
# remove names from data
speaker_specific_data = {k: "\n".join(v) for k, v in speaker_specific_data.items()}
# create corpus as string
corpus = "\n".join(trans)
# remove speaker names from corpus

for k in speakers:
    corpus = corpus.replace(k, "")

Depending on the lenght of your corpus, running the following lines may take a bit

In [ ]:
corpus_words = count_tokens(corpus)
speaker_specific_words = {k: count_tokens(v) for k, v in speaker_specific_data.items()}

Once the results are ready, build the csv file

In [ ]:
csv_rows = [csv_header]
for token in corpus_words.keys():
    csv_row = [token, corpus_words[token]]
    for sp in speakers:
        if token in speaker_specific_words[sp]:
            csv_row.append(speaker_specific_words[sp][token])
        else:
            csv_row.append(0)
    csv_rows.append(csv_row)

Save the results and download

In [ ]:
with open(output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(csv_rows)

files.download(output_path)